In [1]:
import pandas as pd
import sqlite3

In [2]:
# # # Code to delete all tables in the database
# # # ONLY RUN THIS TO RESET THE ENTIRE DATABASE

# database_path = '../data/hop_teaming'

# # Open the database connection
# with sqlite3.connect(database_path) as db:
#     # Create a cursor object to execute SQL commands
#     cursor = db.cursor()

#     # Query the names of all tables in the database
#     cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    
#     # Fetch all table names
#     table_names = cursor.fetchall()
#     print(table_names)

#     # Iterate through each table and drop it
#     for table in table_names:
#         cursor.execute(f"DROP TABLE IF EXISTS {table[0]}")
#     db.commit()

In [3]:
database_path = '../data/hop_teaming'

In [4]:
chunksize = 1000000
n = 1

# The following code reads the CSV file in "chunks", processes each chunk and adds to the database
with sqlite3.connect(database_path) as db:
    for chunk in pd.read_csv('../data/DocGraph_Hop_Teaming_2018_Commercial/DocGraph_Hop_Teaming_2018.csv', chunksize=chunksize):
        chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]  # Clean up the column names
        chunk.to_sql('hopteaming', db, if_exists='append', index=False)  # Append the chunk to a hops table
        print("Added " + str(n * chunksize) + " rows")
        n += 1

Added 1000000 rows
Added 2000000 rows
Added 3000000 rows
Added 4000000 rows
Added 5000000 rows
Added 6000000 rows
Added 7000000 rows
Added 8000000 rows
Added 9000000 rows
Added 10000000 rows
Added 11000000 rows
Added 12000000 rows
Added 13000000 rows
Added 14000000 rows
Added 15000000 rows
Added 16000000 rows
Added 17000000 rows
Added 18000000 rows
Added 19000000 rows
Added 20000000 rows
Added 21000000 rows
Added 22000000 rows
Added 23000000 rows
Added 24000000 rows
Added 25000000 rows
Added 26000000 rows
Added 27000000 rows
Added 28000000 rows
Added 29000000 rows
Added 30000000 rows
Added 31000000 rows
Added 32000000 rows
Added 33000000 rows
Added 34000000 rows
Added 35000000 rows
Added 36000000 rows
Added 37000000 rows
Added 38000000 rows
Added 39000000 rows
Added 40000000 rows
Added 41000000 rows
Added 42000000 rows
Added 43000000 rows
Added 44000000 rows
Added 45000000 rows
Added 46000000 rows
Added 47000000 rows
Added 48000000 rows
Added 49000000 rows
Added 50000000 rows
Added 510

In [5]:
# This function finds the taxonomy code for a given row
def get_taxo_code(row):
    code_prefix = 'Healthcare Provider Taxonomy Code_'
    switch_prefix = 'Healthcare Provider Primary Taxonomy Switch_'
    for num in range(1,16):
        if row[switch_prefix+str(num)] == 'Y':
            break
    return row[code_prefix+str(num)]




In [6]:
# This section creates the npps table

chunksize = 1000000
n = 1

with sqlite3.connect(database_path) as db:
    for chunk in pd.read_csv('../data/NPPES_Data_Dissemination_February_2024/nppes_main.csv', dtype=str, chunksize=chunksize):
        taxonomy_cols = [col for col in chunk.columns if 'Taxonomy Code' in col or 'Taxonomy Switch' in col] #gets the column names which have "Taxonomy Code" or "Taxonomy Switch" in them

        chunk = chunk[['NPI', 'Entity Type Code', 'Provider Organization Name (Legal Business Name)', 'Provider Last Name (Legal Name)',
                        'Provider First Name', 'Provider Middle Name', 'Provider Name Prefix Text', 'Provider Name Suffix Text',
                        'Provider Credential Text', 'Provider First Line Business Practice Location Address', 'Provider Second Line Business Practice Location Address',
                        'Provider Business Practice Location Address City Name', 'Provider Business Practice Location Address State Name',
                        'Provider Business Practice Location Address Postal Code'] + taxonomy_cols] # selects the columns we want

        df = chunk.copy() # making a copy of the chunk to avoid "SettingWithCopyWarning" wwarnings
        df["taxo_code"] = chunk.apply(get_taxo_code, axis=1) # making a column called taxo_code which will hold the taxonomy code for that row
        df = df.drop(columns=taxonomy_cols) # dropping the taxonomy columns as we don't need them anymore
        df = df[df.NPI.notnull() & df.taxo_code.notnull()] # drop the rows in which NPI an taxonomy code are null (don't see how these would be useful)
        df.columns = [x.lower().replace(' ', '_') for x in df.columns]  # Clean up the column names
        df.to_sql('npi', db, if_exists='append', index=False)  # Append the chunk to an npps table
        print("Added " + str(n * chunksize) + " rows")
        n += 1

Added 1000000 rows
Added 2000000 rows
Added 3000000 rows
Added 4000000 rows
Added 5000000 rows
Added 6000000 rows
Added 7000000 rows
Added 8000000 rows
Added 9000000 rows


In [7]:
# This section creates the code_to_classification table
taxonomy_code_to_classification= pd.read_csv('../data/nucc_taxonomy_240.csv', dtype=str,
                          nrows = 100)
with sqlite3.connect(database_path) as db:
    taxonomy_code_to_classification.to_sql('taxonomy', db, if_exists='append', index=False) 

In [8]:
# This section creates the zip_to_cbsa table
chunksize = 10000
n = 0

with sqlite3.connect(database_path) as db:
    for df in pd.read_csv('../data/ZIP_CBSA_122023.csv', dtype=str, chunksize=chunksize):
        df = df.drop(columns=['RES_RATIO','BUS_RATIO', 'OTH_RATIO', 'TOT_RATIO'])
        df = df[df.ZIP.notnull() & df.CBSA.notnull()]
        df.to_sql('zip_cbsa', db, if_exists='append', index=False)  # Append the chunk to the zip_to_cbsa table
        print("Added " + str(df.shape[0]) + " rows")
        n += df.shape[0]

Added 10000 rows
Added 10000 rows
Added 10000 rows
Added 10000 rows
Added 7598 rows


In [9]:
# SCRATCH 

# hopteaming = pd.read_csv('../data/DocGraph_Hop_Teaming_2018_Commercial/DocGraph_Hop_Teaming_2018.csv', 
#                           nrows = 100)


# chunk = pd.read_csv('../data/NPPES_Data_Dissemination_February_2024/nppes_main.csv', dtype=str,
#                           nrows = 100)
# taxonomy_cols = [col for col in chunk.columns if 'Taxonomy Code' in col or 'Taxonomy Switch' in col]

# chunk = chunk[['NPI', 'Entity Type Code', 'Provider Organization Name (Legal Business Name)', 'Provider Last Name (Legal Name)', 
#              'Provider First Name', 'Provider Middle Name', 'Provider Name Prefix Text', 'Provider Name Suffix Text', 
#              'Provider Credential Text', 'Provider First Line Business Practice Location Address', 'Provider Second Line Business Practice Location Address',
#              'Provider Business Practice Location Address City Name', 'Provider Business Practice Location Address State Name', 
#              'Provider Business Practice Location Address Postal Code']+taxonomy_cols]
# df = chunk.copy()
# df["taxo_code"] = chunk.apply(get_taxo_code, axis=1)
# df = df.drop(columns = taxonomy_cols)
# df =df[df.NPI.notnull() & df.taxo_code.notnull()]
# df.columns = [x.lower().replace(' ', '_') for x in df.columns] 